    https://cloud.tencent.com/developer/article/1162834
    https://github.com/ZiJianZhao/SeqGAN-PyTorch
    
    https://blog.csdn.net/Young_Gy/article/details/76474939  构建聊天机器人：检索、seq2seq、RL、SeqGAN
    
    https://blog.csdn.net/yinruiyang94/article/details/77675586 SeqGAN——对抗思想与增强学习的碰撞
    
    https://blog.csdn.net/qunnie_yi/article/details/80129851 只知道GAN你就OUT了——VAE背后的哲学思想及数学原理
    
    https://blog.csdn.net/GitChat/article/details/79081190 手把手教你写一个中文聊天机器人
    
    https://blog.csdn.net/tMb8Z9Vdm66wH68VX1/article/details/79184714 一文读懂智能对话系统